In [1]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import inspect, os, sys

try :
    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

from bs4 import BeautifulSoup
import requests
from requests.compat import quote
from mako.template import Template
from PIL import Image, ImageFile
from openpyxl import load_workbook

import calendar
from io import BytesIO
import re

from modules.wmtools import is_commons_file, get_hash

In [2]:
commons_site = pb.Site("commons", "commons")

In [3]:
cwd = os.getcwd()

images_directory = os.path.join(cwd, 'images')
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

In [4]:
EXCEL_FILE = "diario.madrid.es.xlsx"
wb = load_workbook(os.path.join(cwd, EXCEL_FILE))

In [5]:
template = u"""=={{int:filedesc}}==
{{Information
|description={{es|1=${description}}}
|date=${date}
|source=[${url} Diario de Madrid - ${title}]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Files uploaded by User:Discasto]]
[[Category:Cavalcade of Magi in Madrid in 2018]]
[[Category:${month} ${year} in Madrid]]"""

In [7]:
onlyfiles = [os.path.join(images_directory, f) for f in os.listdir(images_directory) if os.path.isfile(os.path.join(images_directory, f))]
for i, image in enumerate(onlyfiles):
    print (i)
    sha = get_hash(image)
    if is_commons_file (sha) == False:
        print (image)
    else :
        #print (image)
        os.remove(image)

0
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\428 millones usuarios eligieron EMT en 2017 01.jpg
1
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\ALAS llega a los Centros de Mayores del Distrito de Moncloa-Aravaca 01.jpg
2
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Aprobado el proyecto de integración del estadio Santiago Bernabéu en su entorno urbano 01.jpg
3
[[commons:File:Carlos Granados, director de la nueva Oficina Municipal contra el Fraude y la Corrupción 01.jpg]]
4
[[commons:File:Carlos Granados, director de la nueva Oficina Municipal contra el Fraude y la Corrupción 02.jpg]]
5
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Cinco finalistas de los proyectos de viviendas para mayores y jóvenes en Vallecas 01.jpg
6
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Ciudad Lineal rinde homenaje a su vecino más internacional, el astronauta Pedro Duque 01.jpg
7
D:\Datos\Miguel-Angel\Code\Python\co

D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Madrid referente de moda 03.jpg
54
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Madrid se suma a la Declaración de Sevilla y se compromete con la economía circular 01.jpg
55
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Mañana jueves se desactiva el Protocolo de contaminación 01.jpg
56
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Mercado de Productores apuesta por el consumo responsable en Matadero Madrid 01.jpg
57
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Oposiciones para aumentar la plantilla de Policía Municipal en 99 plazas 01.jpg
58
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Oposiciones para aumentar la plantilla de Policía Municipal en 99 plazas 02.jpg
59
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Rita Maestre participa en la inauguración del XII Encuentro de la Unión de Ciudades Capitales Iberoamer

In [8]:
used_names = []

ws = wb[u'Sheet']
for row in ws.iter_rows():
    image_path = row[0].value
    image_name = image_path.split('\\')[-1]
    print (image_name)
    # The image does not exist. Skipping
    if os.path.isfile(image_path) == False :
        print ("*Image name ({0}) deleted from local directory".format(image_name))
        continue

    # If the image name is already in commons, find a new name
    if pb.Page(commons_site, image_name, ns=6).exists():
        print ("**Image name ({0}) already used in Commons".format(image_name))
        used_names.append(image_name)

    while True:
        if image_name in used_names :
            # Finding a new name
            image_subject = '.'.join(image_name.split('.')[:-1])
            image_extension = 'jpg'
            p = re.compile('(.*) ([0-9]{2}\.jpg)')
            m = p.match(image_name)
            if m is None:
                image_name = image_subject + ' 01.' + image_extension
            else :
                counter = int(m.group(2)[:2]) + 1
                image_name = '{} {:02d}.{}'.format(m.group(1), counter, image_extension)

            if pb.Page(commons_site, image_name, ns=6).exists():
                print ("Image name ({0}) already used in Commons. Finding a new name".format(image_name))
                used_names.append(image_name)
        else :
            print ("Preparing to upload image with name {0}".format(image_name))
            used_names.append(image_name)
            break

    # image upload
    vars = {
            "url": row[3].value,
            "description": row[7].value,
            "year": row[5].value,
            "month": row[6].value,
            "date": row[4].value,
            "title": row[2].value
            }
    t = Template(template)
    _text = t.render(**vars)

    bot = UploadRobot([image_path],
                      description = _text,
                      useFilename = image_name,
                      keepFilename = True,
                      verifyDescription = False,
                      ignoreWarning = True,
                      targetSite = commons_site)
    bot.run()

    os.remove(image_path)

Naves Matadero estrena Mystery Magnet, de la artista belga Miet Warlop 01.jpg
*Image name (Naves Matadero estrena Mystery Magnet, de la artista belga Miet Warlop 01.jpg) deleted from local directory
Madrid se suma a la Declaración de Sevilla y se compromete con la economía circular 01.jpg
Preparing to upload image with name Madrid se suma a la Declaración de Sevilla y se compromete con la economía circular 01.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=El Pleno del Ayuntamiento de Madrid ha aprobado por unanimidad de todos los grupos municipales su incorporación a la “Declaración de Sevilla: el compromiso de las ciudades por la economía circular” por el que se asumirá los pronunciamientos y compromisos de dicha declaración que busca construir una Economía Circular en la que los residuos y el uso de recursos se reduzcan, así como que se puedan volver a utilizar después de su vida útil. De esta forma Madrid se suma a otras ciudades como Milán, Londres o Bruselas que buscan poner remedio a los problemas ambientales. Además se compromete a llevar a cabo las exigencias derivadas del paquete de medidas de economía circular aprobado por el Parlamento Europeo.}}
|date=2018-01-31
|source=[https://diario.madrid.es/blog/notas-de-prensa/madrid-se-suma-a-la-declaracion-de-sevilla-y-se-compromete-con-la-economia-circular/ Diario de Madrid - Madrid se suma a la Dec

Carlos Granados, director de la nueva Oficina Municipal contra el Fraude y la Corrupción 01.jpg
*Image name (Carlos Granados, director de la nueva Oficina Municipal contra el Fraude y la Corrupción 01.jpg) deleted from local directory
Carlos Granados, director de la nueva Oficina Municipal contra el Fraude y la Corrupción 02.jpg
*Image name (Carlos Granados, director de la nueva Oficina Municipal contra el Fraude y la Corrupción 02.jpg) deleted from local directory
El Ayuntamiento favorece la promoción interna de los agentes de movilidad 01.jpg
Preparing to upload image with name El Ayuntamiento favorece la promoción interna de los agentes de movilidad 01.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=El Pleno municipal celebrado hoy, miércoles 31 de enero, ha aprobado la modificación puntual del Reglamento de Agentes de Movilidad para favorecer la promoción interna y la carrera profesional de estos funcionarios dentro de este Cuerpo, con el apoyo de Ahora Madrid, PSOE y Ciudadanos, y la abstención del PP.}}
|date=2018-01-31
|source=[https://diario.madrid.es/blog/notas-de-prensa/el-ayuntamiento-favorece-la-promocion-interna-de-los-agentes-de-movilidad/ Diario de Madrid - El Ayuntamiento favorece la promoción interna de los agentes de movilidad]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Files uploaded by User:Discasto]]
[[Category:Cavalcade of Magi in Madrid in 2018]]
[[Category:January 2018 in Madrid]]
Uploading file to commons:commons...
Sleeping for 5.6 seconds, 2018-01-31 23:43:51
Upload succe

El Ayuntamiento recuperará el Teleférico de Madrid a través de la gestión de EMT 01.jpg
Preparing to upload image with name El Ayuntamiento recuperará el Teleférico de Madrid a través de la gestión de EMT 01.jpg


...

The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=Madrid vuelve a ser el centro de todas las miradas a nivel internacional con la celebración de la 67 edición de la Mercedes-Benz Fashion Week, que se celebra entre el 25 y el 29 de enero con un programa que combina los desfiles de 48 diseñadores y marcas nacionales en IFEMA junto con otras actividades que se desarrollan en toda la ciudad.}}
|date=2018-01-24
|source=[https://diario.madrid.es/blog/notas-de-prensa/la-moda-al-alcance-de-toda-la-ciudadania/ Diario de Madrid - La moda al alcance de toda la ciudadanía]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Files uploaded by User:Discasto]]
[[Category:Cavalcade of Magi in Madrid in 2018]]
[[Category:January 2018 in Madrid]]
Uploading file to commons:commons...
Sleeping for 6.9 seconds, 2018-01-31 23:54:30
Upload successful.
Upload of La moda al alcance 

El número de multas de tráfico crece un 0,76% en 2017 01.jpg
Preparing to upload image with name El número de multas de tráfico crece un 0,76% en 2017 01.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=Durante el año 2017, el Ayuntamiento de Madrid formuló un total de 2.955.598 denuncias por infracciones de circulación y estacionamiento, lo que ha supuesto un incremento de 22.372 denuncias, un 0,76% con respecto al mismo periodo de 2016 (2.933.226), según el balance provisional realizado por el Área de Medio Ambiente y Movilidad, en un contexto en que se registra también un leve incremento en el tráfico rodado que, a modo de ejemplo, en la M30 creció un 0,32% en 2017 en términos interanuales.}}
|date=2018-01-24
|source=[https://diario.madrid.es/blog/notas-de-prensa/el-numero-de-multas-de-trafico-crece-un-076-en-2017/ Diario de Madrid - El número de multas de tráfico crece un 0,76% en 2017]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Files uploaded by User:Discasto]]
[[Category:Cavalcade of Magi in M

Mañana jueves se desactiva el Protocolo de contaminación 01.jpg
Preparing to upload image with name Mañana jueves se desactiva el Protocolo de contaminación 01.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=El Ayuntamiento de Madrid desactivará mañana jueves, 25 de enero, el Protocolo de medidas por alta contaminación de dióxido de nitrógeno (NO2), pero para hoy miércoles 24 de enero se mantienen las limitaciones de velocidad máxima a 70 km/hora en la M-30 y en las vías de acceso a la ciudad, en ambos sentidos.}}
|date=2018-01-24
|source=[https://diario.madrid.es/blog/notas-de-prensa/manana-jueves-se-desactiva-el-protocolo-de-contaminacion/ Diario de Madrid - Mañana jueves se desactiva el Protocolo de contaminación]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Files uploaded by User:Discasto]]
[[Category:Cavalcade of Magi in Madrid in 2018]]
[[Category:January 2018 in Madrid]]
Uploading file to commons:commons...
Sleeping for 7.1 seconds, 2018-01-31 23:54:50
Upload successful.
Upload of Mañana jueves se d